# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [4]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [6]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result) #Turning the result (JSON format plain text) into a dictionary
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'training page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'capabilities page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [13]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog/ggml-joins-hf'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'endpoints page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Join page', 'url': 'https://huggingface.co/join'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [16]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 2 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-397B-A17B\nUpdated\nabout 21 hours ago\n•\n390k\n•\n970\nzai-org/GLM-5\nUpdated\n11 days ago\n•\n180k\n•\n1.5k\nNanbeige/Nanbeige4.1-3B\nUpdated\n2 days ago\n•\n202k\n•\n763\nnvidia/personaplex-7b-v1\nUpdated\n9 days ago\n•\n539k\n•\n2.18k\nMiniMaxAI/Mini

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning. It is a leading collaboration platform where the machine learning community comes together to create, discover, and innovate on models, datasets, and applications across multiple modalities including text, image, video, audio, and 3D.

Founded as an open-source hub, Hugging Face empowers machine learning engineers, scientists, and users globally by providing:

- Access to over **2 million models**, **500,000+ datasets**, and **1 million+ applications**
- Tools to build and share ML portfolios
- Fast, scalable paid compute services for inference and model training
- A vibrant community that fosters collaboration to develop open and ethical AI solutions

---

## Core Offerings

### Hugging Face Hub
A central repository to host, explore, and experiment with public models, datasets, and applications.

### Models
Access cutting-edge AI models like Qwen3.5, GLM-5, and others updated regularly with active community engagement.

### Datasets
A rich library of datasets covering various domains to facilitate robust model training and benchmarking.

### Spaces
Run and deploy ML applications easily on Hugging Face’s platform with community-sharing features.

### Compute & Inference
Accelerate your machine learning workflows with Hugging Face’s paid compute resources and easy-to-use inference API.

---

## Community & Collaboration

At its core, Hugging Face is a **community-driven platform** where collaboration thrives. Users can:

- Share models and datasets publicly
- Contribute to open-source ML projects
- Explore trending AI apps and demos
- Build and showcase personal ML portfolios for exposure and learning

The platform prioritizes open, ethical AI development and hosts powered discussions, events, and continuous innovation.

---

## Customers & Users

Hugging Face supports a broad spectrum of users including:

- **AI researchers and scientists** seeking easy access to state-of-the-art ML models and datasets
- **Machine learning engineers** looking to deploy, share, and iterate rapidly on production models
- **Developers and startups** building AI-powered applications across industries
- **Enterprises** requiring scalable, reliable ML infrastructure and collaboration tools
- **Educators and students** learning from and contributing to an extensive AI ecosystem

---

## Company Culture

Hugging Face fosters an inclusive, transparent, and fast-paced culture geared towards innovation and community impact. Key cultural values include:

- **Open collaboration:** Building together in an open and ethical manner
- **Empowerment:** Enabling the next generation of AI practitioners worldwide
- **Transparency and trust:** Maintaining openness in tools, data, and research
- **Agility:** Moving fast with practical tools and user-centric design
- **Community first:** Prioritizing users, contributors, and shared success

Hugging Face embraces diversity and encourages contributions from all skill levels, nurturing a vibrant learning environment.

---

## Careers & Opportunities

Hugging Face is growing rapidly and offers exciting career opportunities for those passionate about machine learning and AI. Roles include:

- Machine Learning Engineers & Researchers
- Software Developers & DevOps Engineers
- Data Scientists and AI Specialists
- Developer Relations and Community Managers
- Product Managers and Customer Success roles

Joining Hugging Face means working at the forefront of AI innovation alongside a global community dedicated to building open and responsible technology.

---

## Connect & Get Started

**Explore models, datasets, and applications:** [huggingface.co](https://huggingface.co)  
**Sign up and build your ML portfolio today!**

---

## Brand Colors & Style

Hugging Face brand colors evoke energy and creativity:

- Yellow: #FFD21E
- Orange: #FF9D00
- Gray: #6B7280

The brand's recognizable style and friendly logo emphasize community and accessibility in AI.

---

**Hugging Face** — The AI community building the future of machine learning with open, collaborative, and ethical tools for all. Join us and accelerate your machine learning journey!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a vibrant, collaborative AI and machine learning community dedicated to building the future of artificial intelligence. It provides the premier platform where machine learning enthusiasts, engineers, scientists, and developers come together to share, explore, and experiment with models, datasets, and AI applications.

The company champions an open and ethical AI future by empowering the global ML community with state-of-the-art resources and tools. Central to their mission is the Hugging Face Hub—a hub that hosts over 2 million models and 500,000 datasets publicly available for collaboration and innovation.

---

## Platform Offerings

- **Models:** Access and contribute to 2M+ machine learning models spanning multiple modalities including text, image, video, audio, and 3D.

- **Datasets:** Browse and utilize from a vast collection of 500k+ datasets curated from a diverse range of domains.

- **Spaces:** Discover and create AI applications and demos, with thousands of community-run interactive Spaces showcasing models in action.

- **Compute:** Accelerate machine learning workflows with paid compute services designed to streamline inference and training.

- **Community:** Engage with an active open-source community that champions transparency, education, and ethical advancements in AI.

---

## Company Culture

- **Collaboration First:** Hugging Face thrives as a community platform offering unlimited hosting, sharing, and collaboration on machine learning projects.

- **Open Source Commitment:** The company strongly supports open source software and transparency, providing open access to cutting-edge ML tools and libraries.

- **Learn and Grow:** Hugging Face is dedicated to helping users build their ML portfolios by sharing models and datasets, encouraging learning and professional development.

- **Ethics and Responsibility:** The ethos of building an ethical AI future underpins all projects, encouraging responsible AI innovation across the community.

---

## Who Uses Hugging Face?

- **Machine Learning Engineers & Data Scientists:** To deploy and share models, datasets, and build AI portfolios.

- **Researchers & Academics:** As a collaborative platform for open research and reproducibility.

- **Developers & AI Enthusiasts:** For quick experimentation, access to pre-trained models, and building AI applications.

- **Enterprises:** Companies looking to leverage advanced AI infrastructure and tools for scalable, reliable AI deployments.

---

## Careers and Opportunities

Hugging Face is growing rapidly and is seeking passionate individuals who wish to make a tangible impact on the AI landscape. Careers at Hugging Face offer:

- Collaboration with top AI talent in an open, innovative environment.
- Opportunities to work on prominent open-source projects.
- Roles across engineering, research, product, community, and enterprise functions.
- A culture that fosters learning, creativity, and ethical AI development.

Visit their website and sign up to join the movement driving the future of machine learning.

---

## Get Started with Hugging Face

- **Explore AI Applications:** Try interactive demos and AI apps within the community.

- **Browse Models and Datasets:** Access millions of models and datasets suitable for diverse machine learning tasks.

- **Join the Community:** Collaborate, share your projects, and contribute to open-source AI development.

- **Accelerate ML Projects:** Utilize paid compute services for fast inference and training.

Sign up today at [huggingface.co](https://huggingface.co) and be part of the AI revolution!

---

## Contact & Social

- **Website:** https://huggingface.co
- **Community:** Engage via forums and Spaces on the platform.
- **Docs & Support:** Comprehensive documentation for all tools and APIs.
  
---

*Hugging Face – Building the Future, Together.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>